<a href="https://colab.research.google.com/github/Ashi743/pyspark/blob/main/MLlib_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("Missing").getOrCreate()

In [2]:
training= spark.read.csv("/content/test1.csv", header=True ,inferSchema= True)

In [3]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [10]:
from pyspark.ml.feature import VectorAssembler
featureassembler= VectorAssembler(inputCols=["age","Experience"],
                                  outputCol="Independent age and exp")


In [11]:
output= featureassembler.transform(training)
output.show()

+---------+---+----------+------+-----------------------+
|     Name|age|Experience|Salary|Independent age and exp|
+---------+---+----------+------+-----------------------+
|    Krish| 31|        10| 30000|            [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|             [30.0,8.0]|
|    Sunny| 29|         4| 20000|             [29.0,4.0]|
|     Paul| 24|         3| 20000|             [24.0,3.0]|
|   Harsha| 21|         1| 15000|             [21.0,1.0]|
|  Shubham| 23|         2| 18000|             [23.0,2.0]|
+---------+---+----------+------+-----------------------+



In [13]:
finalized_data =output.select("Independent age and exp", "Salary")
finalized_data.show()

+-----------------------+------+
|Independent age and exp|Salary|
+-----------------------+------+
|            [31.0,10.0]| 30000|
|             [30.0,8.0]| 25000|
|             [29.0,4.0]| 20000|
|             [24.0,3.0]| 20000|
|             [21.0,1.0]| 15000|
|             [23.0,2.0]| 18000|
+-----------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression

train ,test =finalized_data.randomSplit([0.75,0.25])
regressor= LinearRegression(featuresCol= 'Independent age and exp', labelCol='Salary')
regressor= regressor.fit(train)

In [16]:
regressor.coefficients , regressor.intercept

(DenseVector([47.619, 1285.7143]), 13619.047619047727)

In [18]:
pred= regressor.evaluate(test)
pred.predictions.show()

+-----------------------+------+------------------+
|Independent age and exp|Salary|        prediction|
+-----------------------+------+------------------+
|             [23.0,2.0]| 18000|17285.714285714283|
|            [31.0,10.0]| 30000|27952.380952380976|
+-----------------------+------+------------------+



In [19]:
pred.meanAbsoluteError

1380.9523809523707